In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import random
import swc.aeon.io.reader
import swc.aeon.io.api
from aeon.analysis.movies import gridframes
from aeon.io.video import frames
import plotly.express as px
from aeon.schema.schemas import exp02

In [ ]:
fps = 50
roots = ["/ceph/aeon/aeon/data/ingest/AEON3/social0.2",
         "/ceph/aeon/aeon/data/ingest/AEON3/social0.4",
         "/ceph/aeon/aeon/data/ingest/AEON4/social0.2",
         "/ceph/aeon/aeon/data/ingest/AEON4/social0.3",
         "/ceph/aeon/aeon/data/ingest/AEON4/social0.4"]
starts = [pd.Timestamp("2024-02-09T15:00:00"),
          pd.Timestamp("2024-08-28T09:00:00"),
          pd.Timestamp("2024-02-09T15:00:00"),
          pd.Timestamp("2024-06-19T15:00:00"),
          pd.Timestamp("2024-08-28T08:00:00")]
ends = [pd.Timestamp("2024-02-23T13:00:00"),
        pd.Timestamp("2024-09-09T13:00:00"),
        pd.Timestamp("2024-02-23T12:00:00"),
        pd.Timestamp("2024-07-03T13:00:00"),
        pd.Timestamp("2024-09-09T11:00:00")]

plots_per_root = 50
max_attempts_per_plot = 100  # Prevent infinite loops

for root, start, end in zip(roots, starts, ends):
    print("----------------------")
    print(f"Processing root: {root}")
    plots_found = 0
    
    while plots_found < plots_per_root:
        attempts = 0
        
        while attempts < max_attempts_per_plot:
            # Generate random timestamps until we get one between 7am and 7pm
            while True:
                rand_time = start + pd.Timedelta(seconds=random.randint(0, int((end - start).total_seconds())))
                # Check if time is between 7am and 7pm
                if 7 <= rand_time.hour < 19:
                    break
            
            r = swc.aeon.io.reader.Pose(pattern="CameraTop_222*")
            df = swc.aeon.io.api.load(root, r, start=rand_time, end=rand_time+pd.Timedelta(seconds=1/fps/2))
            
            # Check if dataframe is not empty
            if not df.empty:
                print(f"Plot {plots_found+1}/{plots_per_root} - Root: {root}")
                print(f"Time: {rand_time}")
                if attempts > 0:
                    print(f"Found non-empty dataframe after {attempts+1} attempts")
                
                raw_root = root.replace("ingest", "raw")
                vid_data = swc.aeon.io.api.load(raw_root, exp02.CameraTop.Video, 
                                              rand_time, rand_time+pd.Timedelta(seconds=1/fps/2))
                ts = df.index[0]
                fig = px.imshow(gridframes(list(frames(vid_data)), width=1440, height=1080, shape=1))
                
                # Add scatter plot with larger marker size
                scatter_trace = px.scatter(df, x='x', y='y', size_max=15).data[0]
                scatter_trace.marker.size = 6  # Increase marker size
                scatter_trace.marker.color = 'red'  # Make markers more visible
                fig.add_trace(scatter_trace)
                
                # Update layout to make plot bigger and adjust margins
                fig.update_layout(
                    width=1200,           # Larger display width
                    height=1000,          # Larger display height
                    margin=dict(l=20, r=20, t=20, b=20),  # Reduce margins to use more space
                    showlegend=False      # Remove legend for cleaner look
                )
                
                # Show the figure
                fig.show()
                
                plots_found += 1
                break
            
            attempts += 1
            if attempts == max_attempts_per_plot:
                print(f"Warning: Could not find non-empty dataframe for attempt {plots_found+1} on {root} after {max_attempts_per_plot} attempts")
                break
        
        # If we've exhausted all attempts for this plot, move to the next plot anyway
        if attempts == max_attempts_per_plot:
            plots_found += 1
            
    print(f"Completed processing {plots_found} plots for {root}")

In [ ]:
# Plot a timestamp with known issues
fps = 50
target_time = pd.Timestamp("2024-07-03 13:17:53")

# Load pose data (small window around target time)
processed_root = "/ceph/aeon/aeon/data/processed/AEON4/social0.3"
# raw_root = processed_root.replace("ingest", "raw")
raw_root = processed_root.replace("processed", "raw")
r = swc.aeon.io.reader.Pose(pattern="CameraTop_222*")
df = swc.aeon.io.api.load(processed_root, r, 
                         start=target_time-pd.Timedelta("1s"), 
                         end=target_time+pd.Timedelta("1s"))

# Find closest timestamp
closest_time = df.index[abs(df.index - target_time).argmin()]
df_at_time = df.loc[closest_time:closest_time]

# Load video frame
vid_data = swc.aeon.io.api.load(raw_root, exp02.CameraTop.Video, 
                              closest_time, closest_time + pd.Timedelta(seconds=1/fps/2))
vid_data = vid_data[closest_time:closest_time]

# Create and show plot
fig = px.imshow(gridframes(list(frames(vid_data)), width=1440, height=1080, shape=1))
fig.add_trace(px.scatter(df_at_time, x='x', y='y').data[0])
fig.update_layout(width=1200, height=1000, title=f"Data at {closest_time}")
fig.show()